In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
import time
import tracemalloc
import optuna


In [27]:
datasdp = pd.read_csv('D:\AAAAKK SKRIPSWEETT\Bismillah Code Skripsi V2\Dataset\dataset_sdp_undersampling.csv', delimiter=";")

X = datasdp.drop(columns=['bugs'])
y = datasdp['bugs']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_base, X_train_meta, y_train_base, y_train_meta = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

counts_training = y_train.value_counts()
print("data distribiution data training: ", counts_training)
counts_testing = y_train_base.value_counts()
print("data distribiution data testing: ", counts_testing)

data distribiution data training:  bugs
0    593
1    458
Name: count, dtype: int64
data distribiution data testing:  bugs
0    286
1    239
Name: count, dtype: int64


In [36]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

datasdp = pd.read_csv('D:\AAAAKK SKRIPSWEETT\Bismillah Code Skripsi V2\Dataset\dataset_sdp_undersampling.csv', delimiter=";")

X = datasdp.drop(columns=['bugs'])
y = datasdp['bugs']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y)
X_train_base, X_train_meta, y_train_base, y_train_meta = train_test_split(X_train, y_train)

def objective(trial):
    # Parameter space
    n_estimators = trial.suggest_categorical('n_estimators', [100, 50, 40, 30])
    max_depth = trial.suggest_categorical('max_depth', [1, 4, 8])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 10, 5])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Model
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )

    # Cross-validation
    score = cross_val_score(rf_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Random Forest Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for RF:", study.best_params)


[I 2025-04-16 06:32:03,890] A new study created in memory with name: Random Forest Tuning
[I 2025-04-16 06:32:12,194] Trial 0 finished with value: 0.8885535907238549 and parameters: {'n_estimators': 100, 'max_depth': 4, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 0 with value: 0.8885535907238549.
[I 2025-04-16 06:32:15,689] Trial 1 finished with value: 0.8920575945416788 and parameters: {'n_estimators': 40, 'max_depth': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8920575945416788.
[I 2025-04-16 06:32:18,263] Trial 2 finished with value: 0.8923767767104402 and parameters: {'n_estimators': 30, 'max_depth': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 2 with value: 0.8923767767104402.
[I 2025-04-16 06:32:26,088] Trial 3 finished with value: 0.8010839306498114 and parameters: {'n_estimators': 100, 'max_depth': 1, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 2 with value: 0.8923767767104402.
[I 2025-04-1

Best Parameters for RF: {'n_estimators': 30, 'max_depth': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}


In [38]:
def objective(trial):
    # Parameter space
    n_estimators_et = trial.suggest_categorical('n_estimators', [100, 50, 40, 30])
    max_depth_et = trial.suggest_categorical('max_depth', [1, 4, 8])
    min_samples_leaf_et = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])
    criterion_et = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Model
    et_model = ExtraTreesClassifier(
    n_estimators=n_estimators_et,
    max_depth=max_depth_et,
    min_samples_leaf=min_samples_leaf_et,
    criterion=criterion_et)
   
    # Cross-validation
    score = cross_val_score(et_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Extra Trees Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for ET:", study.best_params)

[I 2025-04-16 06:35:42,544] A new study created in memory with name: Extra Trees Tuning


[I 2025-04-16 06:35:46,559] Trial 0 finished with value: 0.8433361216785504 and parameters: {'n_estimators': 50, 'max_depth': 8, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 0 with value: 0.8433361216785504.
[I 2025-04-16 06:35:47,494] Trial 1 finished with value: 0.6464440887810785 and parameters: {'n_estimators': 30, 'max_depth': 1, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8433361216785504.
[I 2025-04-16 06:35:51,545] Trial 2 finished with value: 0.7085658798656678 and parameters: {'n_estimators': 100, 'max_depth': 1, 'min_samples_leaf': 4, 'criterion': 'gini'}. Best is trial 0 with value: 0.8433361216785504.
[I 2025-04-16 06:36:01,996] Trial 3 finished with value: 0.8372268573077882 and parameters: {'n_estimators': 40, 'max_depth': 8, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 0 with value: 0.8433361216785504.
[I 2025-04-16 06:36:32,842] Trial 4 finished with value: 0.7669891019048481 and parameters: {'n_estimators':

Best Parameters for ET: {'n_estimators': 50, 'max_depth': 8, 'min_samples_leaf': 1, 'criterion': 'gini'}


In [30]:
def objective(trial):
    # Parameter space
    n_estimators_adaa = trial.suggest_categorical('n_estimators', [50, 100, 1000])
    learning_rate_adaa = trial.suggest_categorical('learning_rate', [1, 0.1, 0.001, 0.01])
    

    # Model
    adaboost_model = AdaBoostClassifier(
    n_estimators=n_estimators_adaa,
    learning_rate=learning_rate_adaa
    )

    # Cross-validation
    score = cross_val_score(adaboost_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Random Forest Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for RF:", study.best_params)

[I 2025-04-16 05:55:57,015] A new study created in memory with name: Random Forest Tuning
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_

Best Parameters for RF: {'n_estimators': 100, 'learning_rate': 1}


In [45]:
def objective(trial):
    # Parameter space
    n_estimators_gb = trial.suggest_categorical('n_estimators', [100, 50, 500, 1000])
    learning_rate_gb = trial.suggest_categorical('learning_rate', [0.1, 0.001, 0.01])
    min_samples_leaf_gb = trial.suggest_categorical('min_samples_leaf', [1, 10, 5])
    max_depth_gb = trial.suggest_categorical('max_depth', [3, 7, 9])
    
    # Model
    gb_model = GradientBoostingClassifier(
        n_estimators=n_estimators_gb, 
        learning_rate=learning_rate_gb,
        min_samples_leaf=min_samples_leaf_gb,
        max_depth=max_depth_gb
    )

    # Cross-validation
    score = cross_val_score(gb_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Gradient Boosting Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for GB:", study.best_params)

[I 2025-04-16 11:03:55,033] A new study created in memory with name: Gradient Boosting Tuning
[I 2025-04-16 11:04:06,518] Trial 0 finished with value: 0.9229236862423923 and parameters: {'n_estimators': 50, 'learning_rate': 0.1, 'min_samples_leaf': 10, 'max_depth': 9}. Best is trial 0 with value: 0.9229236862423923.
[I 2025-04-16 11:06:50,980] Trial 1 finished with value: 0.9249851569590366 and parameters: {'n_estimators': 500, 'learning_rate': 0.01, 'min_samples_leaf': 10, 'max_depth': 9}. Best is trial 1 with value: 0.9249851569590366.
[I 2025-04-16 11:11:11,791] Trial 2 finished with value: 0.9261900888706889 and parameters: {'n_estimators': 1000, 'learning_rate': 0.01, 'min_samples_leaf': 5, 'max_depth': 7}. Best is trial 2 with value: 0.9261900888706889.
[I 2025-04-16 11:12:26,475] Trial 3 finished with value: 0.8584387204871478 and parameters: {'n_estimators': 500, 'learning_rate': 0.001, 'min_samples_leaf': 5, 'max_depth': 3}. Best is trial 2 with value: 0.9261900888706889.
[I 2

Best Parameters for GB: {'n_estimators': 1000, 'learning_rate': 0.01, 'min_samples_leaf': 5, 'max_depth': 7}


In [41]:
def objective(trial):
    # Parameter space
    max_iter_hgb = trial.suggest_categorical('max_iter', [100, 50, 500, 1000])
    learning_rate_hgb = trial.suggest_categorical('learning_rate', [0.1, 0.001, 0.01])
    min_samples_leaf_hgb = trial.suggest_categorical('min_samples_leaf', [20, 10, 5])
    max_depth_hgb = trial.suggest_categorical('max_depth', [None, 1, 3, 5])
    
    # Model
    hgb_model = HistGradientBoostingClassifier(
    max_iter=max_iter_hgb,
    learning_rate=learning_rate_hgb,
    min_samples_leaf=min_samples_leaf_hgb,
    max_depth=max_depth_hgb)

    # Cross-validation
    score = cross_val_score(hgb_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Histogram Gradient Boosting Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for HGB:", study.best_params)

[I 2025-04-16 10:14:55,447] A new study created in memory with name: Histogram Gradient Boosting Tuning


[I 2025-04-16 10:15:12,062] Trial 0 finished with value: 0.8772203011537872 and parameters: {'max_iter': 1000, 'learning_rate': 0.1, 'min_samples_leaf': 10, 'max_depth': 1}. Best is trial 0 with value: 0.8772203011537872.
[I 2025-04-16 10:15:35,622] Trial 1 finished with value: 0.7654893868870667 and parameters: {'max_iter': 1000, 'learning_rate': 0.001, 'min_samples_leaf': 5, 'max_depth': 1}. Best is trial 0 with value: 0.8772203011537872.
[I 2025-04-16 10:15:38,689] Trial 2 finished with value: 0.8243912519315751 and parameters: {'max_iter': 500, 'learning_rate': 0.01, 'min_samples_leaf': 20, 'max_depth': 1}. Best is trial 0 with value: 0.8772203011537872.
[I 2025-04-16 10:17:00,519] Trial 3 finished with value: 0.9257769761458942 and parameters: {'max_iter': 1000, 'learning_rate': 0.1, 'min_samples_leaf': 5, 'max_depth': 5}. Best is trial 3 with value: 0.9257769761458942.
[I 2025-04-16 10:17:19,944] Trial 4 finished with value: 0.9242863076203307 and parameters: {'max_iter': 100, 'l

Best Parameters for HGB: {'max_iter': 1000, 'learning_rate': 0.1, 'min_samples_leaf': 5, 'max_depth': 5}


In [42]:
def objective(trial):
    # Parameter space
    n_estimators_xgb = trial.suggest_categorical('n_estimators', [100, 50, 500, 1000])
    learning_rate_xgb = trial.suggest_categorical('learning_rate', [0.3, 0.1, 0.001, 0.01])
    max_depth_xgb = trial.suggest_categorical('max_depth', [6, 3, 4, 5])
    
    # Model
    xgb_model = XGBClassifier(
    n_estimators=n_estimators_xgb, 
    learning_rate=learning_rate_xgb,
    max_depth=max_depth_xgb)

    # Cross-validation
    score = cross_val_score(xgb_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="XG Boosting Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for xgb:", study.best_params)

[I 2025-04-16 10:26:13,282] A new study created in memory with name: XG Boosting Tuning
[I 2025-04-16 10:26:18,338] Trial 0 finished with value: 0.9214203451254495 and parameters: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 5}. Best is trial 0 with value: 0.9214203451254495.
[I 2025-04-16 10:26:24,009] Trial 1 finished with value: 0.886955249976816 and parameters: {'n_estimators': 500, 'learning_rate': 0.001, 'max_depth': 5}. Best is trial 0 with value: 0.9214203451254495.
[I 2025-04-16 10:26:46,980] Trial 2 finished with value: 0.9208500618416625 and parameters: {'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 6}. Best is trial 0 with value: 0.9214203451254495.
[I 2025-04-16 10:26:49,948] Trial 3 finished with value: 0.9217478798260723 and parameters: {'n_estimators': 500, 'learning_rate': 0.1, 'max_depth': 6}. Best is trial 3 with value: 0.9217478798260723.
[I 2025-04-16 10:26:50,651] Trial 4 finished with value: 0.9264743750449356 and parameters: {'n_estimators'

Best Parameters for xgb: {'n_estimators': 50, 'learning_rate': 0.3, 'max_depth': 6}


In [44]:
def objective(trial):
    # Parameter space
    n_estimators_cb = trial.suggest_categorical('n_estimators', [1000, 100, 50, 500])
    learning_rate_cb = trial.suggest_categorical('learning_rate', [0.03, 0.001, 0.01, 0.1])
    depth_cb = trial.suggest_categorical('depth', [1, 5, 10])
    min_data_in_leaf_cb = trial.suggest_categorical('min_data_in_leaf', [1, 10])
    
    
    # Model
    cb_model = CatBoostClassifier(
    n_estimators=n_estimators_cb,
    learning_rate=learning_rate_cb,
    depth=depth_cb,
    min_data_in_leaf=min_data_in_leaf_cb,
    )

    # Cross-validation
    score = cross_val_score(cb_model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Cateforical Boosting Tuning")
study.optimize(objective, n_trials=10, timeout=600)

# Tampilkan hasil terbaik
print("Best Parameters for CatBoost:", study.best_params)

[I 2025-04-16 10:49:20,385] A new study created in memory with name: Cateforical Boosting Tuning


0:	learn: 0.6868481	total: 22.9ms	remaining: 22.9s
1:	learn: 0.6796609	total: 34.9ms	remaining: 17.4s
2:	learn: 0.6741783	total: 39.7ms	remaining: 13.2s
3:	learn: 0.6673108	total: 50.7ms	remaining: 12.6s
4:	learn: 0.6623546	total: 56ms	remaining: 11.1s
5:	learn: 0.6566160	total: 59.8ms	remaining: 9.91s
6:	learn: 0.6514655	total: 64.8ms	remaining: 9.19s
7:	learn: 0.6467612	total: 68.2ms	remaining: 8.45s
8:	learn: 0.6415970	total: 72.4ms	remaining: 7.97s
9:	learn: 0.6376182	total: 75.8ms	remaining: 7.5s
10:	learn: 0.6333500	total: 84.4ms	remaining: 7.58s
11:	learn: 0.6292018	total: 87.2ms	remaining: 7.18s
12:	learn: 0.6250039	total: 92.6ms	remaining: 7.03s
13:	learn: 0.6208118	total: 97.3ms	remaining: 6.85s
14:	learn: 0.6164731	total: 101ms	remaining: 6.61s
15:	learn: 0.6117415	total: 107ms	remaining: 6.59s
16:	learn: 0.6083362	total: 111ms	remaining: 6.44s
17:	learn: 0.6059504	total: 115ms	remaining: 6.25s
18:	learn: 0.6022496	total: 120ms	remaining: 6.17s
19:	learn: 0.5978895	total: 12

[I 2025-04-16 10:51:13,592] Trial 0 finished with value: 0.9259499244041685 and parameters: {'n_estimators': 1000, 'learning_rate': 0.01, 'depth': 5, 'min_data_in_leaf': 10}. Best is trial 0 with value: 0.9259499244041685.


0:	learn: 0.6894859	total: 172ms	remaining: 17s
1:	learn: 0.6858015	total: 264ms	remaining: 12.9s
2:	learn: 0.6824345	total: 370ms	remaining: 12s
3:	learn: 0.6789899	total: 461ms	remaining: 11.1s
4:	learn: 0.6756719	total: 550ms	remaining: 10.4s
5:	learn: 0.6722981	total: 614ms	remaining: 9.62s
6:	learn: 0.6689546	total: 677ms	remaining: 9s
7:	learn: 0.6656932	total: 895ms	remaining: 10.3s
8:	learn: 0.6621616	total: 1.01s	remaining: 10.3s
9:	learn: 0.6590888	total: 1.1s	remaining: 9.88s
10:	learn: 0.6560077	total: 1.17s	remaining: 9.49s
11:	learn: 0.6528276	total: 1.23s	remaining: 9.04s
12:	learn: 0.6496574	total: 1.31s	remaining: 8.8s
13:	learn: 0.6465791	total: 1.39s	remaining: 8.51s
14:	learn: 0.6436246	total: 1.46s	remaining: 8.29s
15:	learn: 0.6406558	total: 1.56s	remaining: 8.19s
16:	learn: 0.6382176	total: 1.64s	remaining: 8s
17:	learn: 0.6350698	total: 1.71s	remaining: 7.8s
18:	learn: 0.6322322	total: 1.78s	remaining: 7.61s
19:	learn: 0.6292683	total: 1.87s	remaining: 7.48s
20:

[I 2025-04-16 10:52:23,537] Trial 1 finished with value: 0.8910413531601767 and parameters: {'n_estimators': 100, 'learning_rate': 0.01, 'depth': 10, 'min_data_in_leaf': 10}. Best is trial 0 with value: 0.9259499244041685.


...

949:	learn: 0.0190925	total: 2.35s	remaining: 124ms
950:	learn: 0.0190730	total: 2.36s	remaining: 121ms
951:	learn: 0.0190503	total: 2.36s	remaining: 119ms
952:	learn: 0.0190254	total: 2.36s	remaining: 116ms
953:	learn: 0.0190242	total: 2.36s	remaining: 114ms
954:	learn: 0.0190004	total: 2.37s	remaining: 112ms
955:	learn: 0.0189796	total: 2.37s	remaining: 109ms
956:	learn: 0.0189568	total: 2.37s	remaining: 107ms
957:	learn: 0.0189504	total: 2.37s	remaining: 104ms
958:	learn: 0.0189358	total: 2.38s	remaining: 102ms
959:	learn: 0.0189358	total: 2.38s	remaining: 99.1ms
960:	learn: 0.0189339	total: 2.38s	remaining: 96.6ms
961:	learn: 0.0189191	total: 2.38s	remaining: 94.2ms
962:	learn: 0.0189028	total: 2.38s	remaining: 91.7ms
963:	learn: 0.0189001	total: 2.39s	remaining: 89.2ms
964:	learn: 0.0189001	total: 2.39s	remaining: 86.7ms
965:	learn: 0.0188973	total: 2.39s	remaining: 84.2ms
966:	learn: 0.0188973	total: 2.4s	remaining: 81.7ms
967:	learn: 0.0188772	total: 2.4s	remaining: 79.3ms
968:	

[I 2025-04-16 10:54:18,853] Trial 8 finished with value: 0.8847924693839486 and parameters: {'n_estimators': 50, 'learning_rate': 0.01, 'depth': 10, 'min_data_in_leaf': 1}. Best is trial 7 with value: 0.9293915551484877.


0:	learn: 0.6766941	total: 3.56ms	remaining: 174ms
1:	learn: 0.6632729	total: 6.41ms	remaining: 154ms
2:	learn: 0.6522946	total: 9.19ms	remaining: 144ms
3:	learn: 0.6432998	total: 11.7ms	remaining: 135ms
4:	learn: 0.6370253	total: 14.7ms	remaining: 132ms
5:	learn: 0.6284283	total: 19.1ms	remaining: 140ms
6:	learn: 0.6223641	total: 22.1ms	remaining: 136ms
7:	learn: 0.6156649	total: 25.6ms	remaining: 134ms
8:	learn: 0.6102555	total: 29.8ms	remaining: 136ms
9:	learn: 0.6067047	total: 32.6ms	remaining: 130ms
10:	learn: 0.6014571	total: 35.5ms	remaining: 126ms
11:	learn: 0.5970246	total: 38.3ms	remaining: 121ms
12:	learn: 0.5926460	total: 42.8ms	remaining: 122ms
13:	learn: 0.5908008	total: 46.1ms	remaining: 118ms
14:	learn: 0.5876735	total: 48.6ms	remaining: 113ms
15:	learn: 0.5835356	total: 51.2ms	remaining: 109ms
16:	learn: 0.5796027	total: 55.5ms	remaining: 108ms
17:	learn: 0.5751474	total: 58.5ms	remaining: 104ms
18:	learn: 0.5720900	total: 61.6ms	remaining: 100ms
19:	learn: 0.5689492	t

[I 2025-04-16 10:54:21,620] Trial 9 finished with value: 0.8223932337823235 and parameters: {'n_estimators': 50, 'learning_rate': 0.1, 'depth': 1, 'min_data_in_leaf': 1}. Best is trial 7 with value: 0.9293915551484877.


Best Parameters for CatBoost: {'n_estimators': 1000, 'learning_rate': 0.1, 'depth': 5, 'min_data_in_leaf': 1}


In [ ]:
#create stacking ensemble
import optuna
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier


# Parameter space
        
estimators_base_model = [
    ('randomforest', rf),
    ('extratrees', et_model),
    ('adaboost', adaboost_model),
    ('gradboost', gb_model),
    ('hgboost', hgb_model),
    ('xgboost', xgb_model),
    ('catboost', cb_model), 
]

#stacked model with  base model and logistic
stack_model = StackingClassifier(
estimators=estimators_base_model, 
final_estimator=RandomForestClassifier(),
n_jobs=-1)

# Cross-validation
score = cross_val_score(stack_model, X, y, cv=5, scoring='roc_auc').mean()
return score

# Jalankan study Optuna
study = optuna.create_study(direction='maximize', study_name="Stacking")
study.optimize(objective, n_trials=10, timeout=600)
# Tampilkan hasil terbaik
print("Best Parameters for stack:", study.best_params)

In [ ]:
#create stacking ensemble
import pickle
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

estimators_base_model = [
    ('randomforest', rf_model),
    ('extratrees', et_model),
    ('adaboost', adaboost_model),
    ('gradboost', gb_model),
    ('hgboost', hgb_model),
    ('xgboost', xgb_model),
    ('catboost', cb_model), 
]

#stacked model with  base model and logistic
stack_model = StackingClassifier(
    estimators=estimators_base_model, 
    final_estimator=RandomForestClassifier(), 
    passthrough=False,
    n_jobs=-1)

#train the model
stack_model.fit(X_train, y_train)

# Simpan model ke file
with open("model_stack3.pkl", "wb") as f:
    pickle.dump(stack_model, f)


#get prediction from stack
stack_predictors = stack_model.predict(X_holdout)

#evaluate model
#metrics calculation
roc_auc = roc_auc_score(y_holdout, stack_predictors)
print('Validation roc auc score: ', roc_auc)

accuracy = accuracy_score(y_holdout, stack_predictors)
print('validation accuracy : ', accuracy)

recall = recall_score(y_holdout, stack_predictors)
print('validation recall : ', recall)

f1score = f1_score(y_holdout, stack_predictors)
print('validation f1-score : ', f1score)

precision  = precision_score(y_holdout, stack_predictors)
print('validation precision : ', precision)

#Confussion Metrics of Random Forest
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_holdout, stack_predictors)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()


NameError: name 'rf_model' is not defined